In [211]:
# Imports
import pandas as pd
import re
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import time

import requests
from bs4 import BeautifulSoup as soup
import lxml
#from dateutil import rrule
# needs a py file to import properly in this structure
#from utils.collection_utils import datetime_parse

from itertools import combinations, permutations, chain


### !!! Datetime function needs work

If the month dict makes a replacement, need to tag that as a month and limit my permutations as required

In [274]:
def datetime_parse(x):
      '''
      Parse datetime out of a string
      More functionality should be added as issues encountered
      Currently 
      '''
      # Remove all non-alpha-numeric
      out = re.sub(r'[^0-9a-zA-Z:]+', ' ', x)
      
      ## Remove any starting tag with :
      # str.split('Updated: ', expand=True)
      
      # Remove time 
      #   .str.replace(r'\b(([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9](:[0-5][0-9])?\s?([AaPp][Mm])?)', ' ')
      
      months = ['January', 'February', 'March', 'April', 'May', 'June',
      'July', 'August', 'September', 'October', 'November', 'December', 'Jan', 'Feb', 'Mar', 'Apr', 'May',
      'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
      
      month_dict = {
      'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6',
      'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 
      'December':'12', 'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5',
      'Jun':'6', 'Jul':'7', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'
      }


      # Removing digits longer than 4 long and words not in months
      out = [out.replace(key, value) for key, value in month_dict.items() if key in out][0]
      
      # Removing digits longer than 4 long and words not in months (redundant)
      out = re.sub(r'[0-9]\d{4,}', ' ', out)
      out = re.sub(r'[0-9]+:[0-9]+', ' ', out) # all after : not tested
      
      out = out.split(' ')
      out = [word for word in out if word.isdigit() or word in months]
      
      out = ' '.join(out[:3])
      # Strip loose whitespace

      out = out.strip()

      # Lists for parsing
      month = ['%b', '%m', '%B']
      day = ['%d']
      year = ['%Y', '%y']

      varieties = list(permutations(chain(month,day,year), 3))
      for v in varieties:
            v = ' '.join(v)
            try:
                  date = [datetime.strptime(str(out), v)]# if d != 0 else d for d in out]

                  if date is not None:
                        print('Successfully parsed with format: ' + v)
                        return date
                        break
            except:
                  #print('Failed: ' + v)
                  pass

In [14]:
load_file = 'collection_urls.jsonl'

df = pd.read_json(load_file, convert_dates=True, lines=True, orient='records')

In [15]:
df.head(7)

,site,url
0,CNA_Russia,https://www.cna.org/centers/cna/sppp/rsp/russi...
1,MIT_Technology_Review,https://www.technologyreview.com/
2,Synced,https://syncedreview.com/
3,IEEE_spectrum,https://spectrum.ieee.org/
4,Import_AI,https://us13.campaign-archive.com/home/?u=67bd...
5,ChinAI,https://chinai.substack.com/archive?sort=new


In [16]:
today = datetime.now()

today

datetime.datetime(2022, 4, 19, 12, 31, 19, 131419)

# Import AI

In [9]:
text_list = []
url_list = []

In [11]:
for url in df.url[4:5]:
      print(url)
      response = requests.post(url) #headers=header)

      html = soup(response.text, 'lxml')

      objects = html.find_all('li', class_="campaign")
      for obj in objects:
            print(obj.text)
            print(obj.a['href'])
            text_list.append(obj.text)
            url_list.append(obj.a['href'])

      print('------------')

https://us13.campaign-archive.com/home/?u=67bd06787e84d73db24fb0aa5&id=6c9d98ff2c
04/11/2022 - Import AI 291: Google trains the world's biggest language model; how robots can be smarter about the world; Conjecture, a new AI alignment company
http://eepurl.com/hZjozT
04/05/2022 - Import AI 290: China plans massive models; DeepMind makes a smaller and smarter model; open source CLIP data
http://eepurl.com/hYSyi9
03/28/2022 - Import AI 289: Copyright v AI art; NIST tries to measure bias in AI; solar-powered Markov chains
http://eepurl.com/hYaLin
03/21/2022 - Import AI 288: Chinese researchers try to train 100trillion+ 'brain-scale' models; 33% of AI benchmarks are meaningless.
http://eepurl.com/hXCPbv
03/07/2022 - Import AI 287: 10 exaflop supercomputer; Google deploys differential privacy; humans can outsmart deepfakes pretty well
http://eepurl.com/hWsYwH
02/28/2022 - Import AI 286: Fairness through dumbness; planet-scale AI computing; another AI safety startup appears
http://eepurl.com/

In [13]:
dates = []
for item in text_list:
      date_sequence = datetime_parse(item)
      dates.append(date_sequence[0])

Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y
Successfully parsed with format: %m %d %Y


In [14]:
df_collected = pd.DataFrame(list(zip(url_list, text_list, dates)), 
            columns=['url', 'text', 'date'])

df_collected.head(2)

,url,text,date
0,http://eepurl.com/hZjozT,04/11/2022 - Import AI 291: Google trains the ...,2022-04-11
1,http://eepurl.com/hYSyi9,04/05/2022 - Import AI 290: China plans massiv...,2022-04-05


In [289]:
import time

In [16]:
keyword ='DeepMind'

In [17]:
relevant_text = []

In [18]:
# Gets text for results within timedelta window
# Only prints ones with the keyword

# To Do:
# 1- To function, df and days and keywords as parameters
# 2- Loop for multiple keywords
# 3 - Return data (not just print)

for index, row in df_collected.iterrows():
      if row.date >= today - timedelta(days=30):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('p')#, class_="campaign")
            issue_text = []
            for obj in objects:
                  issue_text.append(obj.text)
                  #print(obj.text)
            issue_text = ' '.join(issue_text)
            print(issue_text)
            print('--------------------------------')
            if keyword in issue_text:
                  relevant_text.append(issue_text)
                  # add other info, url, date etc. to new data
                  print(True)
            else: 
                  print(False)
            time.sleep(5)
      else:
            pass


Fetching: http://eepurl.com/hZjozT
Welcome to Import AI, a newsletter about artificial intelligence. Forward this email to give your chums an AI upgrade. Subscribe here.

 New dataset lets robots learn about the texture and material of objects, as well as their shape:
…Making robots smarter with the ObjectFolder 2.0 dataset…
Stanford and Carnegie Mellon University researchers have built ObjectFolder 2.0, a dataset of 1000 3D models of objects. ObjectFolder 2.0 tries to render the objects' visual textures and material types, as well as their 3D shapes. ObjectFolder 2.0 contains 1,000 high-quality 3D objects collected from online repositories. It also ships with an "implicit neural representation network that renders visual, acoustic, and tactile sensory data all in real-time with state-of-the-art rendering quality". Transfer learning: The point of datasets like ObjectFolder 2.0 is to try and make it easier to do transfer learning; that is, train a robot (or other AI system) in simulatio

In [20]:
len(relevant_text)

3

# Synced

In [35]:
url = "https://syncedreview.com/"

In [36]:
#today = datetime.now()

today

datetime.datetime(2022, 4, 19, 10, 22, 37, 25532)

In [37]:
year = datetime.now().year

In [38]:
month = datetime.now().month

In [40]:
current_url = url + str(year) + '/' +  str(month) + '/'
current_url

'https://syncedreview.com/2022/4/'

In [48]:
title_list = []
url_list = []
summary_list = []
date_list = []

In [71]:
print(current_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}
print(header)

response = requests.post(url, headers=header)

html = soup(response.text, 'lxml')

objects = html.find_all('div',class_= "entry-main") #"meta-main-wrap"
#objects = html.find_all('p')

#print(objects)
for obj in objects:
      title_list.append(obj.h2.text)#.ul.li['href'])
      url_list.append(obj.a['href'])
      summary_list.append(obj.p.text)
      date_list.append(obj.a.text)

#print('------------')

https://syncedreview.com/2022/4/


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


{'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:27.0) Gecko/20121011 Firefox/27.0'}


In [76]:
print(current_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}
print(header)

response = requests.post(url, headers=header)

html = soup(response.text, 'lxml')

objects = html.find_all('div',id= "primary") 

https://syncedreview.com/2022/4/


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36'}


In [152]:
title_list = []
url_list = []
summary_list = []
date_list = []

In [153]:
#print(objects)
for obj in objects:
      dates = obj.find_all(class_='entry-date')
      for d in dates:
            d_parsed = datetime_parse(d.text)
            date_list.append(d_parsed[0])
            #date_list.append(d.text)

      titles = obj.find_all(class_='entry-title')
      for t in titles:
            title_list.append(t.text)
            url_list.append(t.a['href'])

      summaries = obj.find_all(class_='entry-summary')
      for s in summaries:
            summary_list.append(s.text)



Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %Y %m %d
Successfully parsed with format: %

In [154]:
df_collected2 = pd.DataFrame(list(zip(title_list, url_list, date_list, summary_list)), 
            columns=['title', 'url', 'date', 'summary'])

df_collected2.head(2)

,title,url,date,summary
0,Meet DeepDPM: No Predefined Number of Clusters...,https://syncedreview.com/2022/04/18/meet-deepd...,2022-04-18,\n In the new paper DeepDPM: Deep Clustering W...
1,Alibaba’s USI: A Unified Scheme for Training A...,https://syncedreview.com/2022/04/14/alibabas-u...,2022-04-14,\nIn the new paper Solving ImageNet: a Unified...


In [157]:
keyword = 'AlexNet'

In [ ]:
for index, row in df_collected2.iterrows():
      if row.date >= today - timedelta(days=14):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', class_="entry-content")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)
            print(article_text)
            print('-----------')
            if keyword in article_text:
                  relevant_text.append(article_text)
                  # add other info, url, date etc. to new data
                  print(True)
            else: 
                  print(False)
            print('--------------------------------')

            time.sleep(5)
      else:
            pass


# MIT

In [310]:
base_url = 'https://www.technologyreview.com/'

In [309]:
topic_designator = 'topic/'

topic_tags = ['artificial-intelligence'] 
# humans-and-technology
#computing

In [311]:
final_url = base_url + topic_designator + topic_tags[0]

Requests and Selenium Wire attempt

In [352]:
test_link = "https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name=artificial-intelligence&page=1"

In [351]:
# Making a get request
response = requests.get("https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name=artificial-intelligence&page=1")

# print headers of response
print(response.headers)

{'Server': 'nginx', 'Date': 'Tue, 19 Apr 2022 15:06:43 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Robots-Tag': 'noindex', 'Link': '<https://wp.technologyreview.com/wp-json/>; rel="https://api.w.org/"', 'X-Content-Type-Options': 'nosniff', 'Access-Control-Expose-Headers': 'X-WP-Total, X-WP-TotalPages, Link', 'Access-Control-Allow-Headers': 'Authorization, X-WP-Nonce, Content-Disposition, Content-MD5, Content-Type', 'Access-Control-Allow-Origin': 'https://www.technologyreview.com', 'Cache-Control': 'max-age=60', 'Allow': 'GET', 'X-rq': 'hhn2 0 4 9980', 'Content-Encoding': 'gzip', 'Age': '0', 'X-Cache': 'miss', 'Vary': 'Accept-Encoding, Origin', 'Accept-Ranges': 'bytes', 'Strict-Transport-Security': 'max-age=31536000;includeSubdomains;preload'}


In [353]:
def get_response():
    base = test_link
    final_url = base
    #driver = webdriver.Chrome()
    
    with requests.get(final_url) as response:
        #response = requests.get(url, headers=headers)
        page_soup = soup(response.content, 'lxml')
        return page_soup

In [355]:
# for pid in pids:
import json 

response = get_response()

#parse
j_response = json.loads(response.text)
j_response

[{'name': 'teaser-item',
  'config': {'excerpt': 'An MIT Technology Review series investigates how AI is enriching a powerful few by dispossessing communities that have been dispossessed before.</p>\n',
   'permalink': 'https://www.technologyreview.com/2022/04/19/1049592/artificial-intelligence-colonialism/',
   'teaseCTA': 'Read more',
   'title': 'Artificial intelligence is creating a new colonial world order',
   'topic': 'Artificial intelligence',
   'postDate': '5 hours',
   'postFormat': 'full-story',
   'topicLink': 'https://www.technologyreview.com/topic/artificial-intelligence/',
   'themeName': 'search',
   'showImage': True,
   'sponsorTagline': '',
   'sponsorUrl': '',
   'hideFeaturedImage': False,
   'hideFooter': False,
   'hideHeader': False,
   'bodyLayout': 'single_column',
   'storyColorTheme': 'teal',
   'headlineTextColor': '#ffffff',
   'headerBgColor': 'dark',
   'squareLeadImage': 1050258,
   'summaryBullets': 'This story is the introduction to MIT Technology Re

In [356]:
len(j_response)

10

In [ ]:
# get info I need then link to article for body text

# IEEE

### !!! Try also with requests

In [17]:
base_url ='https://spectrum.ieee.org/'

In [36]:
query_url = 'https://spectrum.ieee.org/search/?q='

In [19]:
topic_url = 'https://spectrum.ieee.org/topic/artificial-intelligence/'

In [20]:
# query connector
C = "&"

In [21]:
sort_criteria = "order=newest"

In [22]:
topic_criteria = "topic=artificial-intelligence"

In [26]:
# initialized above
today

datetime.datetime(2022, 4, 19, 12, 31, 19, 131419)

In [28]:
today.date()

datetime.date(2022, 4, 19)

In [30]:
back_to = today.date() - timedelta(days=30)
back_to

datetime.date(2022, 3, 20)

In [33]:
search_terms = ['deepfake']

In [37]:
for term in search_terms:
      final_url = query_url + term + C + sort_criteria
      print(final_url)

https://spectrum.ieee.org/search/?q=deepfake&order=newest


In [295]:
print(final_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.post(final_url, headers=header)

html = soup(response.text, 'lxml')


https://spectrum.ieee.org/search/?q=deepfake&order=newest


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [300]:

title_list = []
url_list = []
date_list = []

# Not fetching one date!

In [301]:
# Works some at least
obj_n =0

objects = html.find_all('div',  class_='section_column')#elid=True)#class_= 'clearfix')
#print(len(objects))

for obj in objects:
      items = obj.find_all('div', id='col-right')
      print(len(items))
      for i in items:

            print(i.h2.text)
            title_list.append(i.h2.text)

            print(i.h2.a['href'])
            url_list.append(i.h2.a['href'])

            date = i.div.span
            print(date)
            try:
                  d_parsed = datetime_parse(date.text)
                  date_list.append(d_parsed[0])
            except:
                  date_list.append('None')


4


        Can Deepfake Tech Train Computer Vision AIs?
    

https://spectrum.ieee.org/synthetic-data-computer-vision
<span class="social-date__text">06 Apr 2022</span>
['06', '4', '2022']
Successfully parsed with format: %m %d %Y


        AI’s 6 Worst-Case Scenarios
    

https://spectrum.ieee.org/ai-worst-case-scenarios
<span></span>


        If 6G Becomes Just 5G+, We’ll Have Made a Big Mistake
    

https://spectrum.ieee.org/6g-semantic-communication
<span class="social-date__text">16 Dec 2021</span>
['16', '12', '2021']
Successfully parsed with format: %d %m %Y


        Watch Out, Wedding Videographers, AI Is Coming for You
    

https://spectrum.ieee.org/ai-video-editing
<span class="social-date__text">02 Nov 2021</span>
['02', '11', '2021']
Successfully parsed with format: %m %d %Y


In [308]:
df_collected3 = pd.DataFrame(list(zip(title_list, url_list, date_list)), 
            columns=['title', 'url', 'date',])

df_collected3.head(4)

,title,url,date
0,\n\n Can Deepfake Tech Train Computer V...,https://spectrum.ieee.org/synthetic-data-compu...,2022-06-04 00:00:00
1,\n\n AI’s 6 Worst-Case Scenarios\n \n,https://spectrum.ieee.org/ai-worst-case-scenarios,None
2,"\n\n If 6G Becomes Just 5G+, We’ll Have...",https://spectrum.ieee.org/6g-semantic-communic...,2021-12-16 00:00:00
3,"\n\n Watch Out, Wedding Videographers, ...",https://spectrum.ieee.org/ai-video-editing,2021-02-11 00:00:00


In [303]:
keyword = 'GANs'

In [305]:
relevant_text = []

In [307]:
for index, row in df_collected3.iterrows():
      if row.date >= today - timedelta(days=14):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', id="col-center")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)
            print(article_text)
            print('-----------')
            if keyword in article_text:
                  relevant_text.append(article_text)
                  # add other info, url, date etc. to new data
                  print(True)
            else: 
                  print(False)
            print('--------------------------------')

            time.sleep(5)
      else:
            pass

Fetching: https://spectrum.ieee.org/synthetic-data-computer-vision
Generative adversarial networks make phony photorealistic images—and now synthetic data, too A recent survey has revealed that 99 percent of computer-vision engineers have had a machine-learning project completely canceled due to insufficient training data. Researchers have already been looking toward synthetic data to fill this breach. In fact, Gartner foresees that by 2024, more than half of data used for AI and analytics projects will be synthetically generated. A research group from the Massachusetts Institute of Technology has explored whether, given that generative models are now capable of producing highly realistic images, they can replace data sets to train computer-vision AI models. In their paper presented at the 2022 International Conference on Learning Representations, the MIT researchers asked whether, “given access only to a trained generative model, and no access to the dataset that trained it, can we le

TypeError: '>=' not supported between instances of 'str' and 'datetime.datetime'

# CNA

In [357]:
base_url = 'https://www.cna.org/centers/cna/sppp/rsp/russia-ai-archive#newsletters'